In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun  2 11:09:55 2018

@author: Marcelo
"""

import numpy as np
import numpy.ma as ma  # masked array
from machine_selection import set_screen_size_by_machine

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

# 2019-03-09
from teste_datas import (Spec)

# 2019-09-16
from gauss_funcs import gaus_fw, gaus_sig

# 2019-07-15
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.classes.html#basics
from numpy.polynomial import Polynomial as P
from numpy.polynomial.polynomial import (polyfit, polyval)

from scipy.signal import (cwt, ricker, find_peaks, find_peaks_cwt, peak_widths)
from scipy.ndimage import label, generate_binary_structure, find_objects  # 2019-09-18

from scipy.optimize import (curve_fit, root, lsq_linear)
from scipy.stats import (iqr)
from scipy.special import (expit)
import base_line_funcs as blf
import spectra_regions_funcs as spreg

# from spectra_regions_funcs import (total_analysis2)
import csv
import pickle
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh import palettes

In [2]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

In [3]:
# constante k: ver no Maxima: (sqrt(3)*6^(3/2))/(2^(3/2)*5^(5/4)*%pi^(1/4));
k = (np.sqrt(3)*6**(3/2))/(2**(3/2)*5**(5/4)*np.pi**(1/4))
k

0.9041542662259198

In [4]:
kfwhm = 2*np.sqrt(2*np.log(2))
kfwhm

2.3548200450309493

In [5]:
# 2019-09-26 Importar calibra´c~ao em efici^encia de arquivo CSV 

In [6]:
# 2019-09-26 Uma hora usarei isso: ler LibreOffice via Pandas!
# pd.read_excel('path_to_file.ods', engine='odf')

In [7]:
effics = pd.read_csv('2019-sep-25 ALMERA Samples CSV.csv',decimal=',',header=None)

In [8]:
effics

,0,1,2
0,46.54,0.008256,0.026447
1,475.35,0.002822,0.045887
2,563.23,0.002291,0.013179
3,569.32,0.002218,0.010851
4,604.70,0.002068,0.006972
5,795.84,0.001672,0.007033
6,801.93,0.001743,0.013259
7,1038.57,0.001614,0.067092
8,1167.94,0.001424,0.037835
9,1365.15,0.001292,0.026036


In [9]:
# 

In [10]:
npeffics = effics.to_numpy()

In [11]:
# ou entaum
npeffics = effics.to_numpy()
engys = npeffics[:,0]
effs = npeffics[:,1]
s_lneffs = npeffics[:,2]
lnengys = np.log(engys)
lneffs = np.log(effs)
lnengys

array([3.84031216, 6.16405137, 6.33368807, 6.34444267, 6.40473247,
       6.67939816, 6.68702132, 6.94560005, 7.06299679, 7.21901959,
       6.49475183, 6.72722811, 6.69799675, 7.06751591, 7.19481216,
       5.89847168, 4.39444915, 5.62184909, 5.71323763, 5.87498691,
       5.95025185, 4.08664836])

In [12]:
lneffs

array([-4.79681507, -5.87030943, -6.07876688, -6.11114939, -6.18117332,
       -6.39373476, -6.35214751, -6.42903971, -6.55428547, -6.65156387,
       -6.14508204, -6.24043886, -6.12659722, -6.59294454, -6.74648713,
       -5.63016033, -4.86627524, -5.69135995, -5.62128125, -5.68987957,
       -5.768321  , -4.85247836])

In [13]:
# lsq_linear: naum usarei por enquanto
# res = lsq_linear(effics, b, bounds=(lb, ub), lsmr_tol='auto', verbose=1)
# usarei 

In [14]:
# https://docs.scipy.org/doc/numpy/reference/routines.polynomials.classes.html

In [15]:
peff_w = polyfit(lnengys,lneffs,5,w=s_lneffs)
peff_w

array([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
       -1.84188202e+00,  6.36705671e-02])

In [16]:
pfull = polyfit(lnengys,lneffs,5,full=True)
pfull

(array([ 1.29110608e+01, -2.11644869e+01,  9.55439800e+00, -2.03942161e+00,
         2.06411683e-01, -8.06276756e-03]),
 [array([0.11121854]),
  6,
  array([2.41600476e+00, 3.99024315e-01, 6.07435640e-02, 3.28130369e-03,
         1.80800009e-04, 3.56022063e-06]),
  4.884981308350689e-15])

In [17]:
pfull_w = polyfit(lnengys,lneffs,5,full=True,w=s_lneffs)
pfull_w

(array([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
        -1.84188202e+00,  6.36705671e-02]),
 [array([8.04219408e-05]),
  6,
  array([2.42095475e+00, 3.70714692e-01, 3.92583140e-02, 2.73583863e-03,
         1.07477000e-04, 3.30990230e-06]),
  4.884981308350689e-15])

In [18]:
P(peff_w)

Polynomial([-3.56316185e+02,  3.24741130e+02, -1.17786615e+02,  2.09925625e+01,
       -1.84188202e+00,  6.36705671e-02], domain=[-1,  1], window=[-1,  1])

In [19]:
polyfit(lnengys,lneffs,3,w=s_lneffs)

array([12.23997754, -9.27243188,  1.65455979, -0.10164261])

In [20]:
# Bokeh graph w/ efficiencies
bokeffics = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="effs",
   # y_range=[-1000.0, 1000.0], title="engys",
   x_axis_label='channels', y_axis_label='counts'
)

# base, lower, upper = [], [], []

base=lneffs
lower = lneffs - s_lneffs
upper = lneffs + s_lneffs

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

# for i, eff in enumerate(list(effs)):
    # year_mpgs = df[df['yr'] == year]['mpg']
    # mpgs_mean = year_mpgs.mean()
    # mpgs_std = year_mpgs.std()
#    lower.append(lneff - mpgs_std)
#    upper.append(eff + mpgs_std)
#    base.append(year)

bokeffics.circle(lnengys, lneffs, legend="cwt peaks", color="navy",size=10)

# bokeffics.add_layout(
#     Whisker(source=source_error, base="base", upper="upper", lower="lower")
# )

# bokpksallscl.line(xs, y0s, color="lightgreen")
# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokeffics)

In [21]:
# 2019-09-28 PAREY AKIH a parte do cahlculo de eficiehncias

In [22]:
# Alguns defs

In [23]:
# Aqui: inihcio da anahlise
# Leitura do espectro

In [24]:
fn2 = '../gamma/spectra/ALMERA-2017-3/ALM-17-3-#5_PreliA.IEC'
# fn2 = '..\Genie_Transfer\Si\SI2019\SI04219.Chn'
spec2 = Spec()

In [25]:
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)

8192


In [26]:
xs = np.linspace(0,spec2.spLngData-1,spec2.spLngData)
y0s = np.asarray(spec2.spCounts)

In [27]:
# AQUI defino as larguras e quantas. Vamos detonar: tinha usado num=100
# 2019-10-01 Menos agora, usarei num=30

In [28]:
# widths = np.linspace(2,13,num=100)
widths = np.linspace(2,13,num=30)
widths

array([ 2.        ,  2.37931034,  2.75862069,  3.13793103,  3.51724138,
        3.89655172,  4.27586207,  4.65517241,  5.03448276,  5.4137931 ,
        5.79310345,  6.17241379,  6.55172414,  6.93103448,  7.31034483,
        7.68965517,  8.06896552,  8.44827586,  8.82758621,  9.20689655,
        9.5862069 ,  9.96551724, 10.34482759, 10.72413793, 11.10344828,
       11.48275862, 11.86206897, 12.24137931, 12.62068966, 13.        ])

In [29]:
teste1 = find_peaks_cwt(y0s, widths)
teste1

array([ 113,  236,  270,  391,  442,  456,  658,  681,  693,  706,  713,
        728,  777,  794,  812,  827,  844,  885,  920,  943,  994, 1007,
       1020, 1058, 1208, 1225, 1311, 1391, 1495, 1591, 1713, 1782, 1969,
       2054, 2076, 2166, 2195, 2303, 2317, 2339, 2379, 2401, 2432, 2466,
       2585, 2675, 2702, 2751, 2830, 2850, 2937, 2953, 3060, 3085, 3287,
       3312, 3370, 3461, 3487, 3531, 3560, 3584, 3646, 3683, 3698, 3760,
       3812, 3891, 3980, 4083, 4160, 4183, 4249, 4291, 4358, 4396, 4477,
       4497, 4506, 4566, 4581, 4615, 4731, 4777, 4884, 4908, 4942, 4969,
       5026, 5090, 5118, 5150, 5167, 5182, 5204, 5229, 5284, 5298, 5329,
       5348, 5391, 5419, 5470, 5508, 5595, 5675, 5744, 5820, 5852, 5914,
       5950, 5989, 6118, 6170, 6204, 6221, 6235, 6273, 6352, 6490, 6523,
       6588, 6605, 6643, 6670, 6742, 6761, 6806, 6980, 7019, 7101, 7134,
       7228, 7338, 7358, 7401, 7412, 7428, 7453, 7506, 7560, 7577, 7648,
       7680, 7742, 7797, 7820, 7924, 7971, 8023, 80

In [30]:
len(teste1)

157

In [31]:
ipks2 = find_peaks_cwt(y0s,widths=widths,wavelet=ricker,max_distances=widths/4,min_snr=3.0,noise_perc=10)
ipks2

array([ 113,  236,  391,  442,  456,  943, 1208, 1225, 1495, 1713, 1782,
       1969, 2054, 2303, 2339, 2432, 2466, 2585, 2702, 2937, 2953, 3085,
       3370, 3560, 3646, 3760, 3891, 3980, 4083, 4249, 4615, 4731, 4884,
       5329, 5419, 5675, 5852, 6118, 6273, 6490, 6980, 7101, 7134, 7648,
       7797, 7820, 8023, 8106])

In [32]:
ipks2 = find_peaks_cwt(y0s,widths=[widths[0]],wavelet=ricker,max_distances=widths/4,min_snr=3.0,noise_perc=10)
ipks2

array([ 109,  237,  380,  392,  443,  456,  944, 1209, 1226, 1495, 1714,
       1782, 1959, 1970, 2055, 2304, 2434, 2467, 2586, 2703, 2953, 3086,
       3370, 3647, 3892, 3980, 4083, 4250, 4732, 4886, 5676, 5851, 5855,
       6273, 6491, 6981, 7102, 7134, 7649])

In [33]:
ipks2 = find_peaks_cwt(y0s,widths=[widths[1]],wavelet=ricker,max_distances=widths/4,min_snr=3.0,noise_perc=10)
ipks2

array([ 109,  237,  380,  391,  443,  456,  944, 1209, 1226, 1495, 1782,
       1959, 1970, 2055, 2303, 2434, 2467, 2586, 2703, 2953, 3086, 3370,
       3561, 3647, 3892, 3980, 4083, 4250, 4614, 4732, 4885, 5330, 5676,
       5852, 6119, 6273, 6491, 6981, 7102, 7134, 7648, 8024])

In [34]:
# min_snr tem que ser menor
# noise_perc tem que ser > 5 senao escapam muitos. Usar 7
ipks2 = find_peaks_cwt(y0s,widths=[widths[1]],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10)
ipks2

array([ 109,  237,  271,  380,  391,  403,  443,  456,  661,  682,  694,
        706,  719,  728,  745,  755,  779,  793,  826,  875,  921,  944,
        994, 1008, 1209, 1226, 1311, 1392, 1495, 1713, 1767, 1782, 1959,
       1970, 1999, 2055, 2077, 2166, 2196, 2212, 2220, 2303, 2340, 2379,
       2402, 2434, 2467, 2586, 2675, 2703, 2780, 2830, 2939, 2953, 3086,
       3312, 3370, 3385, 3487, 3531, 3561, 3585, 3618, 3647, 3673, 3683,
       3699, 3761, 3892, 3980, 4083, 4160, 4182, 4250, 4352, 4359, 4478,
       4508, 4558, 4566, 4581, 4614, 4644, 4653, 4660, 4732, 4777, 4885,
       4908, 4931, 4941, 4969, 5015, 5026, 5090, 5098, 5119, 5169, 5205,
       5267, 5284, 5298, 5330, 5348, 5378, 5418, 5470, 5507, 5526, 5536,
       5595, 5676, 5852, 5914, 5951, 5961, 5989, 6023, 6119, 6171, 6204,
       6221, 6235, 6273, 6477, 6491, 6524, 6588, 6642, 6670, 6733, 6742,
       6761, 6771, 6808, 6981, 7019, 7102, 7134, 7348, 7360, 7381, 7392,
       7402, 7412, 7429, 7454, 7463, 7494, 7530, 75

In [35]:
len(ipks2)

168

In [36]:
# Separando os picos por escala:
# Aqui prestar atencao redobrada:
# Eh onde defini-se os parametros da busca cwt na funcao find_peaks_cwt
pk_per_scal = []
for wid in widths:
    print (wid)
    pk_per_scal.append(find_peaks_cwt
                       (y0s,widths=[wid],wavelet=ricker,max_distances=widths/4,min_snr=1.0,noise_perc=10))
    # pk_per_scal.append(find_peaks_cwt(y0s,widths=[wid],))
    # print(pk_per_scal[-1])

2.0
2.3793103448275863
2.7586206896551726
3.1379310344827585
3.5172413793103448
3.896551724137931
4.275862068965517
4.655172413793103
5.0344827586206895
5.413793103448276
5.793103448275862
6.172413793103448
6.551724137931034
6.93103448275862
7.310344827586206
7.689655172413793
8.068965517241379
8.448275862068964
8.827586206896552
9.206896551724139
9.586206896551724
9.96551724137931
10.344827586206897
10.724137931034482
11.103448275862068
11.482758620689655
11.86206896551724
12.241379310344827
12.620689655172413
13.0


In [37]:
# pk_per_scal: lista de arrays; cada array corresponde a uma escala e contehm
# os canais dos picos detectados na escala.

# pks_cwt_flatten: lista contihnua em que cada elemento eh uma lista de 2 elementos: [pk, width], em que
#   pk: canal do pico 
#   width: "largura" (=escala)

pks_cwt_flatten = []
for isc, pksc in enumerate(pk_per_scal):
    for pk in pksc:
        pks_cwt_flatten.append([pk,widths[isc]])
len(pks_cwt_flatten)

2130

In [38]:
ar1 = np.asarray(pks_cwt_flatten)

In [39]:
# imag1: imagem de pixels binahrios dos picos nas escalas
imag1 = np.zeros((len(widths),len(xs)))
for isc, pksc in enumerate(pk_per_scal):
    for pk in pksc:
        imag1[isc,pk] = 1
imag1

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
np.shape(imag1)

(30, 8192)

In [41]:
stru_diag_also = generate_binary_structure(2,2)
stru_diag_also

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [42]:
# 2019-10-01 Fun,c~ao chave: label: identifica os ridges na imagem

In [43]:
lblimag1, num_feat_imag1 = label(imag1,stru_diag_also)
lblimag1, num_feat_imag1

(array([[  0,   0,   0, ...,   0, 213,   0],
        [  0,   0,   0, ..., 213,   0,   0],
        [  0,   0,   0, ..., 213,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]], dtype=int32), 260)

In [44]:
# Finally, the ridges delimited all over the spectrum
ridges = find_objects(lblimag1)
# ridges

In [45]:
ridges_rectangles = [lblimag1[loc] for loc in ridges]
# ridges_rectangles

In [46]:
# 2019-09-18 Parei Aqqui

In [47]:
# https://stackoverflow.com/questions/29130898/get-start-and-stop-from-a-python-slice-object

In [48]:
# 2019-09-18 Aqui eh hora de calcular cwtmatr

In [49]:
cwtmatr = cwt(y0s, ricker, widths)
cwtmatr

array([[-1.58740510e-13,  6.73699779e-14,  3.26744393e-13, ...,
         1.36858386e+01,  1.63385596e+01,  1.20432432e+01],
       [-1.09476214e-13, -6.73699779e-15,  4.04219867e-14, ...,
         1.76637103e+01,  1.73034365e+01,  9.98293512e+00],
       [-3.92430121e-13, -3.90745872e-13, -2.79585408e-13, ...,
         1.94460245e+01,  1.65696077e+01,  7.94189113e+00],
       ...,
       [ 4.31167859e-13,  0.00000000e+00,  7.41069757e-13, ...,
         1.52797630e+01,  9.02678519e+00,  2.57067646e+00],
       [-1.88635938e-13, -1.34739956e-12, -1.21265960e-12, ...,
         1.42514133e+01,  8.11962219e+00,  1.80573346e+00],
       [-6.73699779e-13,  4.58115850e-13,  1.07791965e-13, ...,
         1.89620534e+01,  1.32258900e+01,  7.21085146e+00]])

In [50]:
np.shape(cwtmatr)

(30, 8192)

In [51]:
np.shape(lblimag1)

(30, 8192)

In [52]:
# Achar o mahximo cwt no retahngulo

In [53]:
# 2019-09-19 Fica muito confuso usar os mahximos nos eixos. Melhor usar indice flatten
# e depois converter para ihndice estrutural (unravel). Deu certo!
#                                                             |
#                                                             |
#                                                             v

In [54]:
# 2019-10-01
# https://stackoverflow.com/questions/497885/python-element-wise-tuple-operations-like-sum


In [55]:
n_side_chans = 11
n_side_centr_ch = 3

In [56]:
xpk = []
ypk = []
centrpk = []
cwtpk = []
nscpk = []
regions_slcs = []
for iridge, ridge in enumerate(ridges):
    lbl_ridge = iridge+1
    nscpk.append( ridge[0].stop - ridge[0].start)
    labeled = lblimag1[ridge]
    cwts_ridge = cwtmatr[ridge]
    retang3 = ma.masked_where(labeled!=lbl_ridge,cwts_ridge)
    maxim3 = np.max(retang3)
    flat_retang_ind = np.argmax(retang3)
    retang_indices = np.unravel_index(flat_retang_ind,np.shape(retang3))
    # print(iridge)
    # print(ridge)
    # print(maxim3)
    # print(retang_indices)
    indices_max_in_ridge = tuple(map(lambda x, y: x + y, retang_indices, (ridge[0].start, ridge[1].start)))
    cwtpk.append(maxim3)
    xpk.append( indices_max_in_ridge[1] )
    ypk.append( widths[indices_max_in_ridge[0]] )
    region_slc = slice(xpk[-1]-n_side_chans, xpk[-1]+n_side_chans+1)
    regions_slcs.append(region_slc)
    # 2019-10-02 C'alculo de centroide
    region_centr = slice(xpk[-1]-n_side_centr_ch, xpk[-1]+n_side_centr_ch+1)
    centrpk.append( np.sum(xs[region_centr]*y0s[region_centr]) / np.sum(y0s[region_centr]))
    # print(indices_max_in_ridge)
    # print(cwtmatr[indices_max_in_ridge])
    # print('==============')

In [57]:
# Vari'aveis importantes definidas at'e aqui

# n_side_chans = 11
# n_side_centr_ch = 3

# nscpk - n'umero de escalas (=pontos) que comp~oe o ridge 
# xpk - canais dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# ypk - escalas dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# cwtpk - coeficiente cwt correspondente a (xpk, ypk)
# centrpk - centroides dos xpk
# regions_slcs - slices das regi~oes dos picos

In [58]:
# 2019-10-02 Selecionando picos vahlidos pelo critehrio seguinte:
# nsca_det: nuhmero de escalas sucessivas em que eh detectado (= nuhmero de pontos do ridge)
# Supondo que naum haja bifurca,caum, eh a altura do retahngulo que o contehm.
# Adotarei 5 por enquanto
# ou 7

In [59]:
nsca_det = 5
nscpk_ma = ma.masked_less(nscpk, nsca_det)
nscpk_ma

masked_array(data=[10, --, 17, 14, 12, 30, --, 20, 8, 22, --, 6, 8, 7, --,
                   --, 5, --, 7, --, 5, 30, 11, 5, --, --, 11, 30, 6, --,
                   30, --, 8, --, --, 30, --, 8, 22, --, --, 30, --, --,
                   10, 5, --, --, 30, --, 30, --, 12, 7, 30, 30, --, 30,
                   5, 30, --, --, --, 9, 14, 30, --, --, 30, --, --, 5,
                   --, --, 30, 30, --, 12, --, 16, --, 30, 30, 30, 5, 6,
                   --, 30, --, 5, --, --, 7, --, --, --, --, --, --, 26,
                   --, --, 30, --, --, --, 23, 17, 5, --, --, --, --, --,
                   23, --, 9, 11, 8, --, --, 6, --, 30, --, --, --, --,
                   30, --, 7, --, --, --, 12, --, 30, --, --, --, 30, --,
                   --, --, --, 25, --, 18, --, --, --, --, --, 30, --, 30,
                   --, 6, --, --, --, 7, --, 18, 5, --, --, 30, --, 10,
                   --, 30, 11, 25, 30, --, --, --, 7, --, 30, --, 6, 20,
                   --, --, --, --, --, --, --, 

In [60]:
kfwhm = 2*np.sqrt(2*np.log(2))
kfwhm

2.3548200450309493

In [61]:
centrpk

[110.97961824652216,
 151.0116001364722,
 236.19706003289474,
 270.078853046595,
 379.6696919847978,
 390.99176790417124,
 402.62230818826566,
 442.0686249292659,
 455.79453310329956,
 656.014657980456,
 681.9905080960357,
 693.9981903727833,
 705.996300863132,
 717.9911764705882,
 727.992130115425,
 756.0011471648639,
 792.9911356414116,
 811.9902209344441,
 826.9554537121907,
 874.9427510746385,
 921.0024096385542,
 943.1624408549519,
 993.98629121897,
 1007.0053688606084,
 1037.9669169404035,
 1059.9580287668002,
 1208.832788122798,
 1225.3269111987456,
 1311.0160883892227,
 1391.0754269449715,
 1494.9869128911585,
 1591.917280917281,
 1713.0233848592945,
 1726.993670886076,
 1767.0638464551507,
 1781.9764931565246,
 1900.9831223628692,
 1958.0984493328524,
 1965.3411042944786,
 1998.9964249233913,
 2047.0102040816328,
 2053.987158243579,
 2086.0280140070035,
 2092.9980089596816,
 2164.998041136141,
 2194.9658744667886,
 2211.990053050398,
 2219.9383697813123,
 2302.1627430910953,
 

In [62]:
# 2019-10-02 Parei aqui.
# Agora, montar um array com todos os picos ajustados; e subtrair de xs.
# Plotar para ver como fica. Deve ficar uma linha base com alguns picos n~ao subtra'idos.
# Prosseguindo:

In [63]:
# Vari'aveis importantes definidas at'e aqui

# xs - canais
# y0s - contagens
# widhts - escalas

# pk_per_scal: lista de arrays; cada array corresponde a uma escala e contehm os canais dos picos detectados na escala.
# pks_cwt_flatten: lista contihnua em que cada elemento eh uma lista de 2 elementos: [pk, width].

# nscpk - n'umero de escalas (=pontos) que comp~oe o ridge 
# xpk - canais dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# ypk - escalas dos picos ap'os an'alise dos ridges, com coeficiente cwt m'ahximo
# cwtpk - coeficiente cwt correspondente a (xpk, ypk)
# centrpk - centroides dos xpk
# regions_slcs - slices das regi~oes dos picos
# nsca_det: nuhmero m'inimo de escalas sucessivas em que eh detectado (= nuhmero de pontos do ridge)
# nscpk_ma - nscpk com m'ascara para valores < nsca_det 

In [64]:
regions_slcs_valid = np.asarray(regions_slcs)[~nscpk_ma.mask]
regions_slcs_valid

array([slice(100, 123, None), slice(225, 248, None),
       slice(259, 282, None), slice(369, 392, None),
       slice(380, 403, None), slice(431, 454, None),
       slice(445, 468, None), slice(645, 668, None),
       slice(683, 706, None), slice(695, 718, None),
       slice(707, 730, None), slice(782, 805, None),
       slice(816, 839, None), slice(910, 933, None),
       slice(932, 955, None), slice(983, 1006, None),
       slice(996, 1019, None), slice(1198, 1221, None),
       slice(1214, 1237, None), slice(1300, 1323, None),
       slice(1484, 1507, None), slice(1702, 1725, None),
       slice(1771, 1794, None), slice(1947, 1970, None),
       slice(1954, 1977, None), slice(2043, 2066, None),
       slice(2154, 2177, None), slice(2184, 2207, None),
       slice(2291, 2314, None), slice(2328, 2351, None),
       slice(2368, 2391, None), slice(2391, 2414, None),
       slice(2422, 2445, None), slice(2456, 2479, None),
       slice(2574, 2597, None), slice(2664, 2687, None),
      

In [65]:
len(xs)

8192

In [66]:
# 2019-10-07
ypkregs = []
fwhms = []
peak_constructed_spectrum = np.zeros(len(xs))
for ipk, pk in enumerate(nscpk_ma):
    if pk:
        print(ipk, pk)
        # ypicoregio = gaus_sig(xs[regions_slcs[ipk2]],sigmamaximo,gau_hei,centroide)ypicoregio
        # subtrair o pica v'alido
        escalmaxima = ypk[ipk]
        maxcwt = cwtpk[ipk]
        centroide = centrpk[ipk]
        sigmamaximo = escalmaxima / np.sqrt(5)
        fwhm = kfwhm * sigmamaximo
        gau_hei = k * maxcwt / np.sqrt(sigmamaximo)
        print(escalmaxima, maxcwt, centroide, sigmamaximo, gau_hei)
        region_start = regions_slcs[ipk].start
        print(region_start)
        ypicoregio = gaus_sig(xs[regions_slcs[ipk]],sigmamaximo,gau_hei,centroide)
        print(ypicoregio)
        ypkregs.append(ypicoregio)
        fwhms.append(fwhm)
        peak_constructed_spectrum[regions_slcs[ipk]] += ypicoregio
peak_constructed_spectrum

0 10
5.413793103448276 1098.390435890713 110.97961824652216 2.421121879085979 638.2501893963286
100
[2.18388517e-02 1.30511674e-01 6.57628141e-01 2.79397735e+00
 1.00086808e+01 3.02302864e+01 7.69873106e+01 1.65313188e+02
 2.99300415e+02 4.56897311e+02 5.88086690e+02 6.38227574e+02
 5.84011289e+02 4.50586717e+02 2.93121041e+02 1.60778165e+02
 7.43564430e+01 2.89948997e+01 9.53314237e+00 2.64278610e+00
 6.17731023e-01 1.21744199e-01 2.02305927e-02]
2 17
4.275862068965517 4607.806721756599 236.19706003289474 1.9122236497239578 3012.7801913884077
225
[1.08048859e-04 2.01423688e-03 2.85647780e-02 3.08163439e-01
 2.52907275e+00 1.57896128e+01 7.49914218e+01 2.70945314e+02
 7.44700511e+02 1.55708244e+03 2.47668922e+03 2.99682490e+03
 2.75855199e+03 1.93166107e+03 1.02898876e+03 4.16984859e+02
 1.28546391e+02 3.01459823e+01 5.37810086e+00 7.29891647e-01
 7.53559925e-02 5.91843858e-03 3.53611552e-04]
3 14
3.896551724137931 3224.6759312000204 270.078853046595 1.7425909066032843 2208.67029466144

array([ 0.        ,  0.        ,  0.        , ..., 10.67891242,
        6.35121142,  3.63665223])

In [67]:
baseline_spectrum = y0s - peak_constructed_spectrum

In [68]:
len(baseline_spectrum)

8192

In [69]:
np.median(baseline_spectrum[100:2000])

551.0

In [70]:
bl_median_smoother = np.zeros(8192)
bl_median_smoother[20:8172] = np.asarray([np.median(baseline_spectrum[i-n_side_chans:i+n_side_chans+1]) for i in range(20,8172) ])

In [71]:
len(bl_median_smoother)

8192

In [72]:
bl_median_smoother

array([0., 0., 0., ..., 0., 0., 0.])

In [73]:
len(ypkregs)

110

In [74]:
len(y0s[xpk])

260

In [75]:
len(nscpk)

260

In [76]:
len(nscpk_ma)

260

In [77]:
y0s_xpk_valid = y0s[xpk][~nscpk_ma.mask]
y0s_xpk_invld = y0s[xpk][nscpk_ma.mask]
centrpk_valid = np.asarray(centrpk)[~nscpk_ma.mask]
centrpk_invld = np.asarray(centrpk)[nscpk_ma.mask]
print(y0s_xpk_valid)
print(y0s_xpk_invld)
print(fwhms)

[  897  3930  3066 14784 25945  7237  3379   788   822   838   830   875
   797   775  6162   794   767  1452  8244   929 16833   411 25456   407
   330   342   318   243   286   278   232   188   292   288   313   148
   207   210   272 14130   499   113   198   185   133   139  1168   310
   342    98   116   223    98   105   159   619   145   113    79    83
    73    75    75    72   104    92    65    94  2078   264    84    93
   757   200    51    62    49    51    43    47   468   126   187   291
    45    51    45    45   232    59    66    79    98    43    15   899
   298    88    52    34    61    38    48   881   100   832    59    25
  5710   290]
[ 834 2745  747  853  903  804  786  667  650  659  355  330  406  282
  296  309  291  311  225  222  202  181  160  142  139  120  112  109
  112  101  117  102  116  113  113  103   96   95  102  112  116  113
  102  113  104   94   86  106   91  140   77   72   70   72   80   73
   76   74   69   78   62   68   75   76   62

In [78]:
len(y0s_xpk_valid)

110

In [79]:
len(y0s_xpk_invld)

150

In [80]:
len(fwhms)

110

In [81]:
peak_constructed_spectrum

array([ 0.        ,  0.        ,  0.        , ..., 10.67891242,
        6.35121142,  3.63665223])

In [103]:
bokspec = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5], title="Espectro gama",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='canais', y_axis_label='contagens'
)
bokspec.circle(xs, y0s, color="green",size=2)
# bokspec.line(xs, y0s, color="green")
bokspec.line(xs, peak_constructed_spectrum, color="blue")
# bokspec.line(xs, baseline_spectrum, color="orange")
bokspec.line(xs, bl_median_smoother, color="magenta")
bokspec.cross(centrpk_valid, y0s_xpk_valid, color="red",size=16)
bokspec.cross(centrpk_invld, y0s_xpk_invld, color="magenta",size=8)
show(bokspec)

In [83]:
bokspeclin = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokspeclin.circle(xs, y0s, legend="counts", color="green",size=4)
bokspeclin.line(xs, y0s, color="green")
bokspeclin.line(xs, peak_constructed_spectrum, color="blue")
bokspeclin.line(xs, baseline_spectrum, color="orange")
bokspeclin.cross(centrpk_valid, y0s_xpk_valid, color="red",size=16)
bokspeclin.cross(centrpk_invld, y0s_xpk_invld, color="magenta",size=8)
show(bokspeclin)

In [84]:
bokfwhm = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokfwhm.circle(xs, y0s, legend="counts", color="green",size=4)
bokfwhm.line(xs, y0s, color="green")
bokfwhm.line(xs, peak_constructed_spectrum, color="blue")
bokfwhm.line(xs, baseline_spectrum, color="orange")
bokfwhm.cross(centrpk_valid, y0s_xpk_valid, color="red",size=16)
bokfwhm.cross(centrpk_invld, y0s_xpk_invld, color="magenta",size=8)
bokfwhm.circle(centrpk_valid, np.asarray(fwhms)*2000, color="magenta",size=10)
show(bokfwhm)

In [85]:
bokfwhmlog = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokfwhmlog.circle(xs, y0s, legend="counts", color="green",size=4)
bokfwhmlog.line(xs, y0s, color="green")
bokfwhmlog.line(xs, peak_constructed_spectrum, color="blue")
bokfwhmlog.line(xs, baseline_spectrum, color="orange")
bokfwhmlog.cross(centrpk_valid, y0s_xpk_valid, color="red",size=16)
bokfwhmlog.cross(centrpk_invld, y0s_xpk_invld, color="magenta",size=8)
bokfwhmlog.circle(centrpk_valid, np.asarray(fwhms)*2000, color="magenta",size=10)
show(bokfwhmlog)

In [86]:
# 2019-09-13 Numa avaliacao puramente visual, usarei a escala green [5]

In [87]:
bokspecx = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokspecx.circle(xs, y0s, legend="counts", color="lightblue",size=4)
bokspecx.line(xs, y0s, color="lightgreen")
bokspecx.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=16)
bokspecx.cross(pk_per_scal[ 1], y0s[pk_per_scal[ 1]]*1.01**2, color="orange",size=16)
bokspecx.cross(pk_per_scal[ 3], y0s[pk_per_scal[ 3]]*1.01**3, color="brown",size=6)
bokspecx.cross(pk_per_scal[ 5], y0s[pk_per_scal[ 5]]*1.01**4, color="green",size=6)
bokspecx.cross(pk_per_scal[ 8], y0s[pk_per_scal[ 8]]*1.01**5, color="cyan",size=6)
bokspecx.cross(pk_per_scal[12], y0s[pk_per_scal[12]]*1.01**6, color="blue",size=6)
bokspecx.cross(pk_per_scal[15], y0s[pk_per_scal[15]]*1.01**7, color="magenta",size=6)
bokspecx.cross(pk_per_scal[20], y0s[pk_per_scal[20]]*1.01**8, color="violet",size=6)
show(bokspecx)

In [88]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokpksallscl.circle(ar1[:,0], ar1[:,1], legend="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="lightgreen")
# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [89]:
# Bokeh graph w/ peaks in all scales
# 2019-10-01 Now including the cwt maximum in each ridge
bokpksallscl = figure(
   plot_width =default_plot_width,
   # plot_height=default_plot_height,
   plot_height=500,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="synth",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
bokpksallscl.circle(ar1[:,0], ar1[:,1], legend="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="lightgreen")
bokpksallscl.cross(xpk, ypk, color="red",size=12)
show(bokpksallscl)

In [90]:
ar1

array([[1.090e+02, 2.000e+00],
       [1.510e+02, 2.000e+00],
       [2.370e+02, 2.000e+00],
       ...,
       [7.648e+03, 1.300e+01],
       [8.024e+03, 1.300e+01],
       [8.106e+03, 1.300e+01]])

In [91]:
fwhm

13.690398008217628

In [92]:
1 & 1

1

In [93]:
# Coeficientes de a0 a an
# residuals  array...: 

# coef : ndarray, shape (deg + 1,) or (deg + 1, K)
#  Polynomial coefficients ordered from low to high. If y was 2-D, the coefficients in column k of coef
# represent the polynomial fit to the data in y’s k-th column.
# [residuals, rank, singular_values, rcond] : list
# These values are only returned if full = True
# resid – sum of squared residuals of the least squares fit
# rank – the numerical rank of the scaled Vandermonde matrix
# sv – singular values of the scaled Vandermonde matrix
# rcond – value of rcond.
#

In [95]:
# 2019-09-13 PAREi AQUI!!

In [96]:
widths/4

array([0.5       , 0.59482759, 0.68965517, 0.78448276, 0.87931034,
       0.97413793, 1.06896552, 1.1637931 , 1.25862069, 1.35344828,
       1.44827586, 1.54310345, 1.63793103, 1.73275862, 1.82758621,
       1.92241379, 2.01724138, 2.11206897, 2.20689655, 2.30172414,
       2.39655172, 2.49137931, 2.5862069 , 2.68103448, 2.77586207,
       2.87068966, 2.96551724, 3.06034483, 3.15517241, 3.25      ])

In [97]:
# Outro teste: mahximo do cwt no espectro inteiro em cada escala

In [98]:
maxs_escalas = np.max(cwtmatr,axis=1)
print(len(maxs_escalas))
print(maxs_escalas)

30
[25931.0324768  30074.12325138 33616.20949277 36950.72743028
 39315.74479899 40897.97041785 41874.57915532 42395.27708054
 42579.19193726 42517.98332848 42546.74197462 42458.29551199
 42223.71910536 41882.80481787 41465.85220307 40995.8456224
 40490.15143705 39961.88596254 39420.90278123 38907.5127376
 38482.94414174 38042.36870472 37591.98457993 37136.61308909
 36679.99301433 36225.02244215 35773.90325155 35328.32693
 34889.55556144 34505.2778318 ]


In [99]:
print(np.argmax(maxs_escalas))
print(np.max(maxs_escalas))
widths[np.argmax(maxs_escalas)]

8
42579.19193725838


5.0344827586206895

In [100]:
# Entao a soma maxima eh na escala 5.10, entre 4.48 e 5.72. Interessante. Visualmente, achei melhor pegar
# essa mesmo.
# 2019-09-17 Era, jah naum eh mais...
# Agora eh 5.71, entre 5.45 e 5.98

In [101]:
xpk

[111,
 151,
 236,
 270,
 380,
 391,
 403,
 442,
 456,
 656,
 682,
 694,
 706,
 718,
 728,
 756,
 793,
 812,
 827,
 875,
 921,
 943,
 994,
 1007,
 1038,
 1060,
 1209,
 1225,
 1311,
 1391,
 1495,
 1592,
 1713,
 1727,
 1767,
 1782,
 1901,
 1958,
 1965,
 1999,
 2047,
 2054,
 2086,
 2093,
 2165,
 2195,
 2212,
 2220,
 2302,
 2317,
 2339,
 2367,
 2379,
 2402,
 2433,
 2467,
 2499,
 2585,
 2675,
 2702,
 2719,
 2781,
 2831,
 2937,
 2953,
 3086,
 3286,
 3312,
 3370,
 3385,
 3413,
 3487,
 3522,
 3531,
 3561,
 3645,
 3674,
 3683,
 3698,
 3760,
 3813,
 3891,
 3980,
 4083,
 4160,
 4183,
 4241,
 4249,
 4352,
 4359,
 4490,
 4497,
 4507,
 4516,
 4558,
 4566,
 4573,
 4581,
 4596,
 4614,
 4653,
 4660,
 4731,
 4777,
 4871,
 4881,
 4884,
 4909,
 4941,
 4969,
 4991,
 5004,
 5015,
 5026,
 5092,
 5098,
 5117,
 5167,
 5205,
 5250,
 5268,
 5284,
 5298,
 5329,
 5349,
 5366,
 5378,
 5390,
 5419,
 5425,
 5470,
 5508,
 5526,
 5536,
 5596,
 5614,
 5675,
 5758,
 5795,
 5820,
 5852,
 5855,
 5914,
 5941,
 5961,
 5990,
 